In [80]:
using SymPy, NLOptControl, JuMP, Ipopt, FastGaussQuadrature

In [84]:
n = NLOpt();
const g = 1.62519; # m/s^2
function MoonLander{T<:Any}(n::NLOpt,x::Array{T,2},u::Array{T,2}) # dynamic constraint equations
  if n.integrationMethod==:tm
    L = size(x)[1];
  else
    L = size(x)[1]-1;
  end
  dx = Array(Any,L,n.numStates)
  dx[:,1] =  @NLexpression(mdl, [j=1:L], x[j,2] );
  dx[:,2] =  @NLexpression(mdl, [j=1:L], u[j,1] - g);
  return dx
end

n = define(n,stateEquations=MoonLander,numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],XL=[-Inf,-Inf],XU=[Inf,Inf],CL=[0.],CU=[3.])
n = configure(n,Ni=2,Nck=[5,6];(:integrationMethod => :ps),(:integrationScheme => :lgrExplicit),(:finalTimeDV => false),(:tf => 4.0))
mdl = Model(solver = IpoptSolver()); 
@variable(mdl, 0.01 <= tf <= Inf) 
n.tf = tf;
n = create_intervals(mdl,n,tf);

In [3]:
function interval(tf,Ni::Int64,Nck::Array{Int64,1},τ::Array{Vector{Float64},1},ω::Array{Vector{Float64},1})
  di = (tf)/Ni; # interval size
  # create mesh points
    tm = Array(Any,Ni+1); tm[1] = 0;
  for idx in 1:Ni
    tm[idx+1] = tm[idx] + di;
  end
    # go through each mesh interval creating time intervals; [t(i-1),t(i)] --> [-1,1]
    ts=[[scale_tau(τ[int],tm[int],tm[int+1]);di*int] for int in 1:Ni];
    ωₛ=[scale_w(ω[int],tm[int],tm[int+1]) for int in 1:Ni];
    return ts, ωₛ
end

interval (generic function with 1 method)

In [4]:
tf = Sym("tf")
ts, ws = interval(tf,n.Ni,n.Nck,n.τ,n.ω);

In [5]:
ts

1-element Array{Array{SymPy.Sym,1},1}:
 
[          0          ]
[                     ]
[0.0162247659013998*tf]
[                     ]
[0.0536972999397246*tf]
[                     ]
[0.110657191180484*tf ]
[                     ]
[0.184610260556525*tf ]
[                     ]
[0.272323547110735*tf ]
[                     ]
[0.369963311629596*tf ]
[                     ]
[0.473262138660127*tf ]
[                     ]
[ 0.57770534269243*tf ]
[                     ]
[0.678728256011064*tf ]
[                     ]
[0.771915729350742*tf ]
[                     ]
[0.853195132318786*tf ]
[                     ]
[0.919014500318045*tf ]
[                     ]
[0.966498595467987*tf ]
[                     ]
[0.993583239207182*tf ]
[                     ]
[         tf          ]

In [43]:
int=1
test = Array(Any,n.Nck[int]+1)
val = 4; # if this is how it works, would have to change every time..or scale to -1 to 1
for i in 1:n.Nck[int]+1
    test[i] =  ts[int][i](tf=>val)
end

In [72]:
tic()
D = [Array(Any,n.Nck[int],n.Nck[int]+1) for int in 1:n.Ni];
De = [Array(Any,n.Nck[int],n.Nck[int]+1) for int in 1:n.Ni]
val=4
for int in 1:n.Ni
    for idx in 1:n.Nck[int]+1
        for j in 1:n.Nck[int]
            f = lagrange_basis_poly(ts[int][j], test, n.Nck[int], idx)
            D[int][j,idx] = diff(f,tf)
            De[int][j,idx] = D[int][j,idx](tf=>test[j])
        end
    end
end
toc()

elapsed time: 8.298036652 seconds


8.298036652

In [63]:
polyDiff(test)

16×16 Array{SymPy.Sym,2}
[      -28.25          38.3619494684511     -15.6569302097589    9.22749942121
[                                                                             
[ -6.18902831756102    -0.12706154380328    9.01170366054022     -4.3431359668
[                                                                             
[ 1.38442203909697     -4.93909056700484   -0.132093039565496    5.33059222907
[                                                                             
[-0.553142089784207    1.61374275066062     -3.61381780023442   -0.14055322510
[                                                                             
[ 0.285687133690967   -0.779795885858886     1.354824145899      -2.9128221835
[                                                                             
[-0.170866899493002    0.452350132769052   -0.715747582630633    1.17556634778
[                                                                             
[ 0.112400039062441   -0.29

In [74]:
De[1]

15×16 Array{Any,2}:
                    0                    0  …                   0
   -0.449366535202281    0.605719551950218     0.0573070360886304
    -1.21409092678155     1.50242084848525     0.0906452177030752
    -1.07517135680769    0.605841000871969     -0.146629710867730
    0.188533646452601    -1.59071579847435     0.0116057888222612
   0.0330801611683022  -0.0551251312891655  …   0.112882604788693
  -0.0364764236660185   0.0937975659215586     -0.153724239441497
 -0.00129643068730762  0.00619719112823434     0.0667107287641971
   0.0387011133052212   -0.102132687932764      0.218531833211391
  -0.0539196511278705    0.139281977392071     -0.655503058254770
   0.0341433708965674  -0.0872962103696400  …   0.822631841847969
  0.00529286775969728  -0.0135566903954710     0.0119762928588962
  -0.0255535133003069   0.0648785271855310      -1.94569099773056
  0.00952214207834101  -0.0241141204771829       2.87351813741052
   0.0112919449171970  -0.0285861417320935       5.92439

In [82]:
# divide the interval to [-1,1]
taus_and_weights = [gaussradau(n.Nck[int]) for int in 1:n.Ni];
n.τ = [taus_and_weights[int][1] for int in 1:n.Ni];
n.ω = [taus_and_weights[int][2] for int in 1:n.Ni];

In [83]:
function create(mdl::JuMP.Model,n::NLOpt)
  T = linspace(-1,1,n.Ni+1)
  di = (n.tf - n.t0)/n.Ni; # interval size
  # create mesh points
  tm = zeros(Float64,n.Ni+1); tm[1] = t0;
  for idx in 1:n.Ni
    tm[idx+1] = tm[idx] + di;
  end
    # go through each mesh interval creating time intervals; [t(i-1),t(i)] --> [-1,1]
    ts=[[scale_tau(τ[int],tm[int],tm[int+1]);di*int] for int in 1:n.Ni];
    ωₛ=[scale_w(ω[int],tm[int],tm[int+1]) for int in 1:n.Ni];
    return ts, ωₛ
end


1-element Array{Array{Float64,1},1}:
 [-1.0,-0.96755,-0.892605,-0.778686,-0.630779,-0.455353,-0.260073,-0.0534757,0.155411,0.357457,0.543831,0.70639,0.838029,0.932997,0.987166]

In [85]:
  T = linspace(-1,1,n.Ni+1)


3-element LinSpace{Float64}:
 -1.0,0.0,1.0

In [87]:
  di = (n.tf - 0)/n.Ni # interval size


0.5 tf

In [50]:
int=1
j=5
x=ts[int][j]
x_data = test;# ts[int]
Nc = n.Nck[int]
L=Array(Any,Nc+1)
L_previous=1;
for idx in 1:Nc+1 # use all of the data
  if idx!=j # would be /0 error
    L[idx] =  L_previous*(x - x_data[idx])/(x_data[j]-x_data[idx]);
    L_previous = L[idx];
  end
end